# Load Dataset

Nvidia Documentation Question and Answer pairs Q&A dataset for LLM finetuning about the NVIDIA about SDKs and blogs


In [3]:
!pip list

Package                                  Version
---------------------------------------- -----------
accelerate                               0.29.3
aiofiles                                 23.2.1
aiohttp                                  3.9.5
aiosignal                                1.3.1
annotated-types                          0.6.0
anyio                                    3.7.1
asttokens                                2.4.1
async-timeout                            4.0.3
asyncer                                  0.0.2
attrs                                    23.2.0
bidict                                   0.23.1
certifi                                  2024.2.2
chainlit                                 1.0.504
charset-normalizer                       3.3.2
chevron                                  0.14.0
click                                    8.1.7
comm                                     0.2.2
dataclasses-json                         0.5.14
debugpy                                  

## This is important for import other packages from another directory

In [1]:
%cd ..

/workspaces/Chatchat_AIMeng/src


In [2]:
from utils.constants import RANDOM_SEED
from utils.util_funcs import data_cleaning
import pandas as pd

from datasets import load_dataset, Dataset

/workspaces/Chatchat_AIMeng/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
from datasets import load_dataset

dataset = load_dataset("ajsbsd/nvidia-qa")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'question', 'answer'],
        num_rows: 7108
    })
})

# Split dataset and save


In [31]:
dataset = dataset['train']
dataset

Dataset({
    features: ['Unnamed: 0', 'question', 'answer'],
    num_rows: 7108
})

In [34]:
dataset_split = dataset.train_test_split(test_size=0.2, seed=RANDOM_SEED)
dataset_split

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'question', 'answer'],
        num_rows: 5686
    })
    test: Dataset({
        features: ['Unnamed: 0', 'question', 'answer'],
        num_rows: 1422
    })
})

In [35]:
train_dataset = dataset_split['train']
test_dataset = dataset_split['test']
train_dataset

Dataset({
    features: ['Unnamed: 0', 'question', 'answer'],
    num_rows: 5686
})

In [36]:
train_val_dataset = train_dataset.train_test_split(test_size=0.2, seed=RANDOM_SEED)
train_val_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'question', 'answer'],
        num_rows: 4548
    })
    test: Dataset({
        features: ['Unnamed: 0', 'question', 'answer'],
        num_rows: 1138
    })
})

In [37]:
train_dataset = train_val_dataset['train']
val_dataset = train_val_dataset['test']
test_dataset = test_dataset # original

In [38]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'question', 'answer'],
    num_rows: 4548
})

In [39]:
test_dataset

Dataset({
    features: ['Unnamed: 0', 'question', 'answer'],
    num_rows: 1422
})

In [40]:
val_dataset

Dataset({
    features: ['Unnamed: 0', 'question', 'answer'],
    num_rows: 1138
})

In [41]:
train_df = pd.DataFrame(train_dataset)
test_df = pd.DataFrame(test_dataset)
val_df = pd.DataFrame(val_dataset)

In [42]:
train_df.head()

,Unnamed: 0,question,answer
0,5189,What limitation does cuBLAS-XT solve for GPU a...,cuBLAS-XT overcomes the limitation of manually...
1,4339,What is the role of the NVIDIA CUDA compiler i...,The NVIDIA CUDA compiler optimizes memory reso...
2,1398,What are the two ways to modify behavior in th...,The two ways to modify behavior in the system ...
3,2457,How does the cudaMemAdvise API enhance memory ...,The cudaMemAdvise API provides memory usage hi...
4,2157,What kind of measurement is used to compare fl...,The comparison of floating-point results betwe...


In [43]:
# remove useless columns
train_df.drop(columns=['Unnamed: 0'], inplace=True)
test_df.drop(columns=['Unnamed: 0'], inplace=True)
val_df.drop(columns=['Unnamed: 0'], inplace=True)

In [44]:
train_df.to_csv('data/raw/train.csv', index=False)
test_df.to_csv('data/raw/test.csv', index=False)
val_df.to_csv('data/raw/val.csv', index=False)

In [49]:
pd.read_csv('./data/processed/train.csv').index

RangeIndex(start=0, stop=4548, step=1)

In [53]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  4548 non-null   object
 1   answer    4548 non-null   object
dtypes: object(2)
memory usage: 71.2+ KB


# Post Processing with Data

## 1. Data Clean

In [55]:
train_df['question'] = train_df['question'].apply(data_cleaning)
val_df['question'] = val_df['question'].apply(data_cleaning)
test_df['question'] = test_df['question'].apply(data_cleaning)

train_df['answer'] = train_df['answer'].apply(data_cleaning)
val_df['answer'] = val_df['answer'].apply(data_cleaning)
test_df['answer'] = test_df['answer'].apply(data_cleaning)

In [56]:
train_df

,question,answer
0,what limitation does cublasxt solve for gpu ac...,cublasxt overcomes the limitation of manually ...
1,what is the role of the nvidia cuda compiler i...,the nvidia cuda compiler optimizes memory reso...
2,what are the two ways to modify behavior in th...,the two ways to modify behavior in the system ...
3,how does the cudamemadvise api enhance memory ...,the cudamemadvise api provides memory usage hi...
4,what kind of measurement is used to compare fl...,the comparison of floatingpoint results betwee...
...,...,...
4543,what is the connection between the lowpower gk...,the lowpower gk208 gpu present in the geforce ...
4544,how does unified memory impact the development...,unified memory simplifies memory management in...
4545,how can loop unrolling improve cuda code perfo...,loop unrolling is a technique that reduces loo...
4546,which digitized manuscripts were used for the ...,the study conducted by the university of notre...


In [57]:
train_df.to_csv('data/processed/train.csv', index=False)
test_df.to_csv('data/processed/test.csv', index=False)
val_df.to_csv('data/processed/val.csv', index=False)

# 2. Tokenization

In [3]:
train_df = pd.read_csv('data/processed/train.csv')
test_df = pd.read_csv('data/processed/test.csv')
val_df = pd.read_csv('data/processed/val.csv')

In [4]:
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)
test_data = Dataset.from_pandas(test_df)

In [5]:
train_data

Dataset({
    features: ['question', 'answer'],
    num_rows: 4548
})

In [6]:
%pwd

'/workspaces/Chatchat_AIMeng/src'

In [7]:
from transformers import AutoTokenizer, GenerationConfig
from utils.util_funcs import tokenized_batch_function
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
import os
hf_token = os.getenv('HF_TOKEN')

In [9]:
train_tokenized = train_data.map(lambda x: tokenized_batch_function(x, hf_token = hf_token), batched=True)
val_tokenized = val_data.map(lambda x: tokenized_batch_function(x,hf_token = hf_token), batched=True)
test_tokenized = test_data.map(lambda x: tokenized_batch_function(x,hf_token = hf_token), batched=True)

Map: 100%|██████████| 1422/1422 [00:01<00:00, 1288.12 examples/s]


In [12]:
train_tokenized

Dataset({
    features: ['question', 'answer', 'input_ids', 'labels'],
    num_rows: 4548
})

In [13]:
train_tokenized = train_tokenized.select_columns(["input_ids", "labels"])
val_tokenized = val_tokenized.select_columns(["input_ids", "labels"])
test_tokenized = test_tokenized.select_columns(["input_ids", "labels"])

In [14]:
train_tokenized

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 4548
})

# After tokenize

In [19]:
train_tokenized['input_ids'][0:1]

[[148,
  33,
  3,
  9,
  11860,
  18,
  188,
  29,
  7,
  3321,
  53,
  6165,
  5,
  2150,
  12,
  8,
  826,
  822,
  10,
  125,
  14130,
  405,
  123,
  4605,
  7,
  226,
  17,
  4602,
  21,
  3,
  122,
  4987,
  23224,
  947,
  33,
  128,
  207,
  4269,
  10,
  3,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [21]:
train_tokenized.save_to_disk('data/processed/train_tokenized')
val_tokenized.save_to_disk('data/processed/val_tokenized')
test_tokenized.save_to_disk('data/processed/test_tokenized')

Saving the dataset (1/1 shards): 100%|██████████| 1422/1422 [00:00<00:00, 114817.32 examples/s]
